In [1]:
import dimod
from dimod.generators.constraints import combinations
from dwave.system import LeapHybridSampler
from hybrid.reference import KerberosSampler

import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize)

from numpy.random import shuffle
import json
import csv
import itertools
import os
import math
import random
import matplotlib.pyplot as plt

from utils import *
from qubos import *

notebook_path = os.path.abspath("main.ipynb")

In [2]:
filename = "factorizations_f2.npz" #list(uploaded.keys())[0]
with open(filename, 'rb') as f:
  factorizations = dict(np.load(f, allow_pickle=True))

u, v, w = factorizations['4,4,4']

x_vectors, y_vectors, z_vectors = vectorize(u), vectorize(v), vectorize(w)

#print(get_standard_tensor(4))

In [3]:
initial = get_standard_tensor_rectangular(4) #np.zeros((4**2, 4**2, 4**2)) #get_standard_tensor(4)
xp, yp, zp = None, None, None
stop = False
for x, y, z in zip(x_vectors[:-46], y_vectors[:-46], z_vectors[:-46]):
    if stop:
        xp, yp, zp = x, y, z
        break
    initial = (initial - np.tensordot(x, np.tensordot(y, z, axes=0), axes=0)) % 2
    print("vectors: ", x, y, z)
    #print(np.array_str(initial))
    non_zeros = np.count_nonzero(initial.flatten())
    print("Number of nonzero: ", non_zeros)
    if non_zeros == 2140:
        stop = True
    print("---------------------------------------------")
#print(initial)
#xp, yp, zp = x_vectors[-2], y_vectors[-2], z_vectors[-2]
initial = get_standard_tensor_rectangular(4)

vectors:  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0] [1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Number of nonzero:  65
---------------------------------------------


In [4]:
bqm = towards_origo(initial, 4)
energies = []
for x, y, z in zip(x_vectors, y_vectors, z_vectors):
    energies.append({ "x": x, "y": y, "z": z, "energy": bqm.energy(construct_point_for_energy_evaluation(initial, x, y, z, 4))})
#print(max(energies))

ordered_energies = sorted(energies, key=lambda d: -d['energy'])

#sample = solve_bqm_in_leap(bqm, sampler ="LeapHybrid")
#print_solution(sample)

initial = get_standard_tensor_rectangular(4)
for elem in ordered_energies[:1]:
    print(elem)
    initial = (initial - np.tensordot(elem["x"], np.tensordot(elem["y"], elem["z"], axes=0), axes=0)) % 2
    #print(initial)
    non_zeros = np.count_nonzero(initial.flatten())
    print("Number of nonzero: ", non_zeros)

{'x': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'y': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], 'z': [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0], 'energy': 406.0}
Number of nonzero:  75


In [5]:
def search_matrix_multiplication(initial_tensor, limit, file_name, dim):
    #tensor1 = initial_tensor
    tensor2 = initial_tensor #- np.tensordot([1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1], np.tensordot([1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1], [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1], axes=0), axes=0)
    file = open("results//" + file_name + ".txt", "w")
    file.write(str(initial_tensor) + "\n")
    file.close()
    for i in range(limit):
        #bqm1 = towards_standard(tensor1, dim)
        bqm2 = towards_origo(tensor2, dim)
        #sample1 = solve_bqm_in_leap(bqm1, sampler ="LeapHybrid")
        #print_solution(sample1)
        sample2 = solve_bqm_in_leap(bqm2, sampler = "Simulated")
        print_solution(sample2)
        #x1, y1, z1 = process_result(sample1, dim)
        #print(x1, y1, z1)
        x2, y2, z2 = process_result(sample2, dim)
        print(x2, y2, z2)
        # len(x1) + len(y1) + len(z1) > 0
        
        if len(x2) + len(y2) + len(z2) > 0:
            #tensor1 = (tensor1 - np.tensordot(x1, np.tensordot(y1, z1, axes=0), axes=0)) % 2
            tensor2 = (tensor2 - np.tensordot(x2, np.tensordot(y2, z2, axes=0), axes=0)) % 2
            file = open("results//" + file_name + ".txt", "a")
            #file.write(str(x1) + " " + str(y1) + " " + str(z1) + "\n")
            #file.write(str(tensor1) + "\n")
            file.write(str(x2) + " " + str(y2) + " " + str(z2) + "\n")
            file.write(str(tensor2) + "\n")
            file.close()
            if np.array_equal(tensor2, np.zeros((dim**2, dim**2, dim**2))):
                print("Algorithm reached origo")
                break
        else:
            break

In [6]:
search_matrix_multiplication(initial, 50, "4x4", 4)

C:\Users\valte\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\function_base.py:423: RuntimeWarning: invalid value encountered in log10
  log_start = _nx.log10(start)


Energy:  225.0
('x0', 'y10') 1
('x0', 'y11') 1
('x0', 'y12') 1
('x0', 'y13') 1
('x0', 'y14') 1
('x0', 'y15') 1
('x0', 'y4') 1
('x0', 'y5') 1
('x0', 'y6') 1
('x0', 'y7') 1
('x0', 'y8') 1
('x0', 'y9') 1
('x0', 'z1') 1
('x0', 'z10') 1
('x0', 'z11') 1
('x0', 'z13') 1
('x0', 'z14') 1
('x0', 'z15') 1
('x0', 'z2') 1
('x0', 'z3') 1
('x0', 'z5') 1
('x0', 'z6') 1
('x0', 'z7') 1
('x0', 'z9') 1
('x1', 'y0') 1
('x1', 'y1') 1
('x1', 'y10') 1
('x1', 'y11') 1
('x1', 'y12') 1
('x1', 'y13') 1
('x1', 'y14') 1
('x1', 'y15') 1
('x1', 'y2') 1
('x1', 'y3') 1
('x1', 'y8') 1
('x1', 'y9') 1
('x1', 'z1') 1
('x1', 'z10') 1
('x1', 'z11') 1
('x1', 'z13') 1
('x1', 'z14') 1
('x1', 'z15') 1
('x1', 'z2') 1
('x1', 'z3') 1
('x1', 'z5') 1
('x1', 'z6') 1
('x1', 'z7') 1
('x1', 'z9') 1
('x10', 'y0') 1
('x10', 'y1') 1
('x10', 'y12') 1
('x10', 'y13') 1
('x10', 'y14') 1
('x10', 'y15') 1
('x10', 'y2') 1
('x10', 'y3') 1
('x10', 'y4') 1
('x10', 'y5') 1
('x10', 'y6') 1
('x10', 'y7') 1
('x10', 'z0') 1
('x10', 'z1') 1
('x10', 'z11') 